In [1]:
import networkx as nx
from mtg_draft_ai import synergy, controller

card_list = controller.read_cube_toml('cube_81183_tag_data.toml')
COLOR_PAIRS = ['WU', 'WB', 'WR', 'WG', 'UB', 'UR', 'UG', 'BR', 'BG', 'RG']

In [2]:
def contains_all_colors(cards, colors):
    colors_seen = set()
    for card in cards:
        if card.color_id != 'C':
            for color in card.color_id:
                colors_seen.add(color)
    
    return colors_seen == set(colors)

In [3]:
def default_ratings(card_list):
    defaults = {}

    # Default rating is currently based on centrality, but likely we could find a better measure.
    for color_combo in COLOR_PAIRS:
        defaults[color_combo] = {}
        on_color_cards = [c for c in card_list if synergy.castable(c, color_combo)]
        colors_graph = synergy.create_graph(on_color_cards)

        for c in colors_graph.nodes:
            defaults[color_combo][c] = colors_graph.degree(c)
        """
        comms = nx.community.greedy_modularity_communities(colors_graph)
        for comm in comms:
            if not contains_all_colors(comm, color_combo):
                continue
            comm_subgraph = colors_graph.subgraph(comm)
            comm_centralities = nx.eigenvector_centrality_numpy(comm_subgraph)

            for k, v in comm_centralities.items():
                defaults[color_combo][k] = v
        """
        # Normalize values into 0-1 range
        max_value = max(defaults[color_combo].values())
        for k, v in defaults[color_combo].items():
            defaults[color_combo][k] = v / max_value
        
        # Fill in default rating of 0 for cards not in graph
        for card in on_color_cards:
            defaults[color_combo].setdefault(card, 0)
    
    return defaults

In [4]:
import pprint

defaults = default_ratings(card_list)

pprint = pprint.PrettyPrinter(indent=4)

for color_combo in COLOR_PAIRS:
    print('Colors: {}'.format(color_combo))
    dict_to_print = {k.name: v for k, v in defaults[color_combo].items()}
    pprint.pprint(synergy._sort_dict_by_values(dict_to_print, reverse=True))

Colors: WU
[   ('Glint-Nest Crane', 1.0),
    ('Weatherlight', 0.972972972972973),
    ('Sai, Master Thopterist', 0.918918918918919),
    ('Buried Ruin', 0.8918918918918919),
    ('Piston Sledge', 0.8648648648648649),
    ('Nebelgast Herald', 0.8108108108108109),
    ('Rattlechains', 0.8108108108108109),
    ("Artificer's Assistant", 0.7567567567567568),
    ('Naru Meha, Master Wizard', 0.7567567567567568),
    ('Epiphany at the Drownyard', 0.7297297297297297),
    ('Grateful Apparition', 0.7297297297297297),
    ('Rise from the Tides', 0.7027027027027027),
    ('Soulfire Grand Master', 0.7027027027027027),
    ('Karn, Silver Golem', 0.6756756756756757),
    ('Metalwork Colossus', 0.6756756756756757),
    ('Quickling', 0.6756756756756757),
    ('Skilled Animator', 0.6756756756756757),
    ('Thopter Squadron', 0.6756756756756757),
    ('Mindless Automaton', 0.6486486486486487),
    ('Steel Overseer', 0.6486486486486487),
    ('Essence Flux', 0.6216216216216216),
    ('Anafenza, Kin-Tree

In [5]:
card_list[0]

Card: {'name': 'Abhorrent Overlord', 'color_id': 'B', 'tags': [('Reanimator', 'Payoff'), ('ETB', 'Enabler'), ('Ramp', 'Payoff')], 'power_tier': 2}